In [1]:
#basic imports and env
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('paid_sec_api_key')

In [2]:
import libs.sec_fin_statement_helpers as sec_fin

In [3]:
import requests
import json
import pandas as pd
import libs.sec as sec

In [4]:
# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

Can Pull:

RevenueFromContractWithCustomerExcludingAssessedTax, CostOfGoodsAndServicesSold, GrossProfit, ResearchAndDevelopmentExpense, SellingGeneralAndAdministrativeExpense, OperatingExpenses, OperatingIncomeLoss, NonoperatingIncomeExpense, IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest, IncomeTaxExpenseBenefit, NetIncomeLoss, EarningsPerShareBasic, EarningsPerShareDiluted, WeightedAverageNumberOfSharesOutstandingBasic, WeightedAverageNumberOfDilutedSharesOutstanding

In [5]:
from sec_api import QueryApi
# get your API key at https://sec-api.io
query_api = QueryApi(api_key=api_key)

In [6]:
ticker_list = pd.read_csv("./Data/Cleaned_Data/Ticker_library.csv")["Ticker"].to_list()
ticker_list = ['ABNB','UBER']

In [7]:
def SEC_filings_query(ticker_list,filingTypes:list):
    filings_dict = {}
    for ticker in ticker_list:
        filings=[]
        for fileType in filingTypes:
            query = {
                "query": { "query_string": { 
                    "query": f"ticker:{ticker} AND filedAt:"+"{2012-06-01 TO 2022-06-01} AND formType:\""+f"{fileType}\"" 
                } },
                "from": "0",
                "size": "60",
                "sort": [{ "filedAt": { "order": "desc" } }]
            }
            filings.extend(query_api.get_filings(query)['filings'])
        filings_dict[ticker] = filings
    return filings_dict

In [8]:
all_filings = SEC_filings_query(ticker_list,['10-Q','10-k'])

In [9]:
accession_numbers = {}

# extract accession numbers of each filing
for ticker in ticker_list:
    accessionNums = []
    for filing in all_filings[ticker]:
        accessionNums.append(filing['accessionNo'])
    accession_numbers[ticker] = accessionNums

accession_numbers

{'ABNB': ['0001559720-22-000009',
  '0001559720-21-000017',
  '0001628280-21-016979',
  '0001628280-21-010389',
  '0001559720-22-000006',
  '0001559720-21-000010'],
 'UBER': ['0001543151-22-000015',
  '0001543151-21-000047',
  '0001543151-21-000038',
  '0001543151-21-000029',
  '0001628280-20-015936',
  '0001543151-20-000029',
  '0001543151-20-000022',
  '0001543151-19-000017',
  '0001543151-19-000009',
  '0001628280-19-007524',
  '0001543151-22-000008',
  '0001543151-21-000014',
  '0001543151-20-000010']}

In [10]:
# from IPython.display import display, HTML
def create_final_income_statement(ticker_accession_numbers):
    # helps printing dataframes while we generate new income statement
    previous_income_statement_set = False
    income_statement_final = None

    for accession_no in ticker_accession_numbers:
    # for accession_no in accession_numbers: # doesn't work with filings filed before 2017 - indicies not equal
        print('Processing: ' + accession_no)
        try:
            # get XBRL-JSON of 10-Q or 10-K filing by accession number
            xbrl_json_data = sec_fin.get_xbrl_json(accession_no)
            
            # convert XBRL-JSON to a pandas dataframe
            income_statement_uncleaned = sec_fin.get_income_statement(xbrl_json_data)

            # clean the income statement
            income_statement_cleaned = sec_fin.clean_income_statement(income_statement_uncleaned)
            
            # print income statement on each iteration to monitor progress
            # display(HTML(income_statement_cleaned.to_html()))
            
            # merge new income statement with previously generated income statement
            if previous_income_statement_set:
                income_statement_final = sec_fin.clean_income_statement(sec_fin.merge_income_statements(income_statement_final, income_statement_cleaned))
            else:
                income_statement_final = income_statement_cleaned
                previous_income_statement_set = True
        except:
            continue
    return income_statement_final
# model_statements_final = create_final_income_statement_df(accession_numbers)

In [11]:
def create_final_income_statement_df(accession_numbers_dict):
    df_dict = {}
    for ticker in accession_numbers_dict.keys():
        print(f'Start Processing: {ticker}')
        try:
            ticker_df = create_final_income_statement(accession_numbers[ticker]).transpose().reset_index()
            ticker_df.rename(columns={0:"Dates"})
            ticker_df['Ticker']=ticker #add Column name
            df_dict[ticker] = ticker_df
        except:
            continue
    return df_dict
def create_nonModel_final_income_statement_df(accession_numbers_dict):
    df_dict = {}
    for ticker in accession_numbers_dict.keys():
        print(f'Start Processing: {ticker}')
        try:
            ticker_df = create_final_income_statement(accession_numbers[ticker])
            df_dict[ticker] = ticker_df
        except:
            continue
    return df_dict

In [12]:
income_statements_final = create_final_income_statement_df(accession_numbers)

Start Processing: ABNB
Processing: 0001559720-22-000009
Processing: 0001559720-21-000017
Processing: 0001628280-21-016979
Processing: 0001628280-21-010389
Processing: 0001559720-22-000006
Processing: 0001559720-21-000010
Start Processing: UBER
Processing: 0001543151-22-000015
Processing: 0001543151-21-000047
Processing: 0001543151-21-000038
Processing: 0001543151-21-000029
Processing: 0001628280-20-015936
Processing: 0001543151-20-000029
Processing: 0001543151-20-000022
Processing: 0001543151-19-000017
Processing: 0001543151-19-000009
Processing: 0001628280-19-007524
Processing: 0001543151-22-000008
Processing: 0001543151-21-000014
Processing: 0001543151-20-000010


In [13]:
statements = income_statements_final

In [14]:
statements = [sec_fin.add_fourth_quarter_results(statement) for statement in statements]

IndexError: list index out of range

In [ ]:
qrt_income_statement = statements = [sec_fin.only_quarterly_results(statement) for statement in statements]
qrt_income_statement

,2018-04-01-2018-06-30,2018-06-30-2018-09-29,2018-09-30-2018-12-29,2018-12-30-2019-03-30,2019-03-31-2019-06-29,2019-06-29-2019-09-28,2019-09-29-2019-12-28,2019-12-29-2020-03-28,2020-03-29-2020-06-27,2020-06-27-2020-09-26,2020-09-27-2020-12-26,2020-12-27-2021-03-27,2021-03-28-2021-06-26,2021-06-26-2021-09-25,2021-09-26-2021-12-25,2021-12-26-2022-03-26
RevenueFromContractWithCustomerExcludingAssessedTax,53265000000,62900000000,84310000000,58015000000,53809000000,64040000000,91819000000,58313000000,59685000000,64698000000,111439000000,89584000000,81434000000,83360000000,123945000000,97278000000
CostOfGoodsAndServicesSold,32844000000,38816000000,52279000000,36194000000,33582000000,39727000000,56602000000,35943000000,37005000000,40009000000,67111000000,51505000000,46179000000,48186000000,69702000000,54719000000
GrossProfit,20421000000,24084000000,32031000000,21821000000,20227000000,24313000000,35217000000,22370000000,22680000000,24689000000,44328000000,38079000000,35255000000,35174000000,54243000000,42559000000
ResearchAndDevelopmentExpense,3701000000,3750000000,3902000000,3948000000,4257000000,4110000000,4451000000,4565000000,4758000000,4978000000,5163000000,5262000000,5717000000,5772000000,6306000000,6387000000
SellingGeneralAndAdministrativeExpense,4108000000,4216000000,4783000000,4458000000,4426000000,4578000000,5197000000,4952000000,4831000000,4936000000,5631000000,5314000000,5412000000,5616000000,6449000000,6193000000
OperatingExpenses,7809000000,7966000000,8685000000,8406000000,8683000000,8688000000,9648000000,9517000000,9589000000,9914000000,10794000000,10576000000,11129000000,11388000000,12755000000,12580000000
OperatingIncomeLoss,12612000000,16118000000,23346000000,13415000000,11544000000,15625000000,25569000000,12853000000,13091000000,14775000000,33534000000,27503000000,24126000000,23786000000,41488000000,29979000000
NonoperatingIncomeExpense,672000000,303000000,560000000,378000000,367000000,502000000,349000000,282000000,46000000,126000000,45000000,508000000,243000000,-538000000,-247000000,160000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,13284000000,16421000000,23906000000,13793000000,11911000000,16127000000,25918000000,13135000000,13137000000,14901000000,33579000000,28011000000,24369000000,23248000000,41241000000,30139000000
IncomeTaxExpenseBenefit,1765000000,2296000000,3941000000,2232000000,1867000000,2441000000,3682000000,1886000000,1884000000,2228000000,4824000000,4381000000,2625000000,2697000000,6611000000,5129000000


In [ ]:
for ticker in qrt_income_statement.keys():
    qrt_income_statement[ticker].to_csv(f'.\Data\Cleaned_Data\SEC_Fin_Data\{ticker}_SEC_Fin_data')